In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

2022-06-06 19:44:06.439137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-06 19:44:06.439156: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
csv = pd.read_csv('sheet.csv')

In [3]:
csv.isnull().values.any()

False

In [4]:
drope = ['Invoice ID', 'Tax 5%', 'Date', 'Time', 'cogs', 'gross margin percentage']

In [5]:
df = csv.drop(drope, axis=1)

In [6]:
df

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Total,Payment,gross income,Rating
0,A,Yangon,Member,Female,Health and beauty,74.69,7,548.9715,Ewallet,26.1415,9.1
1,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,80.2200,Cash,3.8200,9.6
2,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,340.5255,Credit card,16.2155,7.4
3,A,Yangon,Member,Male,Health and beauty,58.22,8,489.0480,Ewallet,23.2880,8.4
4,A,Yangon,Normal,Male,Sports and travel,86.31,7,634.3785,Ewallet,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...
995,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,42.3675,Ewallet,2.0175,6.2
996,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,1022.4900,Ewallet,48.6900,4.4
997,A,Yangon,Member,Male,Food and beverages,31.84,1,33.4320,Cash,1.5920,7.7
998,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,69.1110,Cash,3.2910,4.1


In [7]:
minmax = MinMaxScaler()
onehot = OneHotEncoder()

In [8]:
y = df.pop('Total')

In [9]:
colunas = df.columns

In [10]:
for column in colunas:
    if(df[column].dtype == 'object'):
        tmp = pd.DataFrame((onehot.fit_transform(df[[column]])).todense())
        tmp.index = df.index
        df = pd.concat([df, tmp], axis=1)
        df.drop(column, inplace=True, axis=1)
    else:
        df[column] = minmax.fit_transform(df[column].values.reshape(-1,1))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df, y)
ipt_dim = X_train.shape[1]

In [12]:
model = Sequential()
model.add(Dense(512, input_shape=[ipt_dim], activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)

2022-06-06 19:44:10.948584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-06 19:44:10.948604: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-06 19:44:10.948619: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (j): /proc/driver/nvidia/version does not exist
2022-06-06 19:44:10.948897: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model.fit(X_train, y_train, batch_size=32, epochs=150, callbacks=[early_stop], verbose=0)

Epoch 76: early stopping


In [14]:
model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 1ms/step - loss: 27.9144 - mae: 3.9427


[27.914443969726562, 3.9427309036254883]

In [15]:
model.predict(X_test)[:10]

8/8 [==============================] - 0s 1ms/step


array([[368.64764 ],
       [432.64603 ],
       [339.47305 ],
       [323.38998 ],
       [228.72693 ],
       [293.48483 ],
       [ 99.455086],
       [ 73.76728 ],
       [322.10208 ],
       [113.57099 ]], dtype=float32)

In [16]:
y_test[:10]

103    367.0380
6      433.6920
812    341.0925
805    325.3740
559    228.1230
938    290.0835
417     85.5855
66      70.2870
633    314.0550
415    100.9155
Name: Total, dtype: float64